In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("db.sqlite3")
df = pd.read_sql_query("select * from submissions limit 5000;", conn)

In [2]:
df.shape

(5000, 4)

In [3]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import itertools, string, operator, re, unicodedata, nltk
import nltk
from operator import itemgetter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from bs4 import BeautifulSoup
import numpy as np
from itertools import combinations
from gensim.models import Phrases
from collections import Counter
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

'''Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

'''Metrics/Evaluation'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle

'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'gensim'

In [3]:
df.head(10)

,subreddit,subscribers,title,text
0,AskReddit,26241971,"People who haven't pooped in 2019 yet, why are...",
1,AskReddit,26241971,Stan Lee has passed away at 95 years old,As many of you know today is day that many of ...
2,Home,10022,Beautiful Home :),
3,Home,10022,My roommate is kicking me out because having a...,"So, I am not asking for advice, really...mores..."
4,videos,22077202,This is what happens when one company owns doz...,
5,videos,22077202,Youtube is Facilitating the Sexual Exploitatio...,
6,news,19777808,Blizzard Employees Staged a Walkout After the ...,
7,news,19777808,Kobe Bryant killed in helicopter crash in Cali...,
8,worldnews,23055673,Trump Impeached for Abuse of Power,
9,worldnews,23055673,"Two weeks before his inauguration, Donald J. T...",


In [4]:
df['clean_text'] = df['text'].str.replace('[^\w\s]',' ')
df['clean_title'] = df['title'].str.replace('[^\w\s]',' ')

In [5]:
df['text'] = df['clean_text'] + df['clean_title']
df['text'].shape

(2000,)

In [60]:
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_md")

tokenizer = Tokenizer(nlp.vocab)
tokens = []

""" Make them tokens """

#stop words
STOP_WORDS = nlp.Defaults.stop_words.union(['', ' ', '-', 'reddit', 'post'])
    
tokens = []

for doc in tokenizer.pipe(df['text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())
   
    tokens.append(doc_tokens)
    
df['tokens'] = tokens

In [61]:
def get_lemmas(text):
        """Return the Lemmas"""
        lemmas = []
        doc = nlp(text)
    
        for token in doc: 
            if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
                lemmas.append(token.lemma_)
    
        return lemmas

In [51]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=get_lemmas, min_df=0.025, max_df=.98, ngram_range=(1,2))

In [52]:
#Creating the features (tf-idf weights) for the processed text


#texts = df['text'].astype('str')


#tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), 
                                 #  min_df = 1, 
                                  # max_df = .95)

X = tfidf_vectorizer.fit_transform(df.clean_title) #features
y = df['subreddit'].values #target

print(X)
print(X.shape)

  (0, 18)	0.5443666240588995
  (0, 0)	0.2376844387160356
  (0, 14)	0.4999591067430192
  (0, 10)	0.630247552781986
  (1, 18)	1.0
  (2, 1)	1.0
  (3, 0)	1.0
  (5, 13)	0.7028671489613855
  (5, 0)	0.7113211447095422
  (9, 0)	1.0
  (10, 0)	1.0
  (12, 0)	1.0
  (13, 13)	0.5969383165261838
  (13, 1)	0.27095478536211615
  (13, 0)	0.7551477673622926
  (16, 0)	1.0
  (17, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 5)	0.6693253601901151
  (21, 1)	0.4961108460908987
  (21, 0)	0.5530620133378762
  (22, 2)	1.0
  (26, 15)	0.6071279601134142
  :	:
  (978, 18)	0.45831084480916146
  (980, 15)	0.6762041021169061
  (980, 1)	0.49193407087891017
  (980, 0)	0.5484057641826644
  (981, 13)	0.5501110219881119
  (981, 0)	0.8350915300056606
  (984, 0)	1.0
  (985, 13)	0.8922704661800722
  (985, 0)	0.45150129034455333
  (986, 7)	0.5301810675789723
  (986, 8)	0.5301810675789723
  (986, 6)	0.522188428558498
  (986, 0)	0.4063684488752159
  (989, 0)	1.0
  (990, 5)	1.0
  (992, 0)	1.0
  (993, 0)	1.0
  (994, 0)	

In [57]:
#dimensionality reduction
lsa = TruncatedSVD(n_components=10, 
                   n_iter=10, 
                   random_state=3)

X = lsa.fit_transform(X)
X.shape

(1000, 10)

In [58]:
#Preliminary model evaluation using default parameters

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format


#Creating a dict of the models
model_dict = {'Dummy' : DummyClassifier(random_state=3),
              'Stochastic Gradient Descent' : SGDClassifier(random_state=3, loss='log'),
              'Random Forest': RandomForestClassifier(random_state=3),
              'AdaBoost': AdaBoostClassifier(random_state=3),
              'Gaussian Naive Bayes': GaussianNB(),
              'K Nearest Neighbor': KNeighborsClassifier()}

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .5, 
                                                    shuffle = True, 
                                                    stratify = y, 
                                                    random_state = 3)

#Function to get the scores for each model in a df
def model_score_df(model_dict):   
    model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in model_dict.items():   
        model_name.append(k)
        v.fit(X_train, y_train)
        y_pred = v.predict(X_test)
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

model_score_df(model_dict)

,model_name,accuracy_score,precision_score,recall_score,f1_score
4,Gaussian Naive Bayes,0.01,0.00,0.01,0.00
2,Random Forest,0.00,0.00,0.00,0.00
0,Dummy,0.00,0.00,0.00,0.00
5,K Nearest Neighbor,0.01,0.00,0.01,0.00
3,AdaBoost,0.00,0.00,0.00,0.00
1,Stochastic Gradient Descent,0.00,0.00,0.00,0.00
